In [10]:
import numpy as np
from PIL import Image


def create_random_grayscale(mu=128, sigma=32, size=256, squares=8):
    # Create array for the pattern
    pattern = np.zeros((size, size), dtype=np.uint8)
    square_size = size // squares

    # Generate random grayscale values for each square
    for i in range(squares):
        for j in range(squares):
            # Random value between 0 (black) and 255 (white)
            gray_value = int(np.clip(np.random.normal(mu, sigma), 0, 255))
            pattern[
                i * square_size : (i + 1) * square_size,
                j * square_size : (j + 1) * square_size,
            ] = gray_value

    # Convert to image and save
    img = Image.fromarray(pattern)
    img.save(
        "/home/luanagbmartins/Documents/CEIA/offline_to_online/mujoco_playground/mujoco_playground/_src/locomotion/go1/xmls/assets/moutainfield.png"
    )


# Create a 256x256 image with 8x8 random grayscale squares
create_random_grayscale(128, 200, 256, 128)

In [11]:
# 50, 125, 200

In [12]:
from etils import epath
from mujoco_playground._src import mjx_env

In [13]:
ROOT_PATH = mjx_env.ROOT_PATH / "locomotion" / "go1"
FEET_ONLY_FLAT_TERRAIN_XML = ROOT_PATH / "xmls" / "scene_mjx_feetonly_flat_terrain.xml"
FEET_ONLY_ROUGH_TERRAIN_XML = (
    ROOT_PATH / "xmls" / "scene_mjx_feetonly_rough_terrain.xml"
)
FULL_FLAT_TERRAIN_XML = ROOT_PATH / "xmls" / "scene_mjx_flat_terrain.xml"
FULL_COLLISIONS_FLAT_TERRAIN_XML = (
    ROOT_PATH / "xmls" / "scene_mjx_fullcollisions_flat_terrain.xml"
)

FEET_ONLY_STAIRS_XML = ROOT_PATH / "xmls" / "scene_mjx_feetonly_stairs.xml"
FEET_ONLY_BOWL_XML = ROOT_PATH / "xmls" / "scene_mjx_feetonly_bowl.xml"

In [14]:
import mujoco
import mujoco_viewer


def get_assets():
    assets = {}
    mjx_env.update_assets(assets, ROOT_PATH / "xmls", "*.xml")
    mjx_env.update_assets(assets, ROOT_PATH / "xmls" / "assets")
    path = mjx_env.MENAGERIE_PATH / "unitree_go1"
    mjx_env.update_assets(assets, path, "*.xml")
    mjx_env.update_assets(assets, path / "assets")
    return assets


model = mujoco.MjModel.from_xml_string(
    epath.Path(FEET_ONLY_ROUGH_TERRAIN_XML).read_text(), assets=get_assets()
)
data = mujoco.MjData(model)

# Set initial state to home position
mujoco.mj_resetDataKeyframe(model, data, 0)  # 0 is the index of "home" keyframe

# create the viewer object
viewer = mujoco_viewer.MujocoViewer(model, data)

# simulate and render
for _ in range(10000):
    if viewer.is_alive:
        mujoco.mj_step(model, data)
        viewer.render()
    else:
        break

# close
viewer.close()